In [61]:
import os
import numpy as np
import plotly.plotly as py
import pandas as pd

In [129]:
def process_result_by_country(path_to_scilab_results, parameter_of_interest):
    listOfResults=os.listdir(path_to_scilab_results)
    path_to_reference_file = path_to_scilab_results + str(listOfResults[-2])
    reference_file=open(path_to_reference_file)
    listOfRegions=reference_file.read().split("\n")
    
    #Initialize the dictionary
    countries={}

    for line in listOfRegions:
        if listOfRegions[-1]==line:
            break

        path_to_parameter_file = path_to_scilab_results + \
                                 str(listOfRegions.index(line)+1) + \
                                 parameter_of_interest + ".sci"            

        last_reading = float(open(path_to_parameter_file).read().split("\n")[-2])

        weather_file_name = line.split("/")[-1] 
        country_code = weather_file_name.split("_")[0]

        if country_code in countries:
            countries[country_code].append(last_reading)
        else:
            countries[country_code] = [last_reading]

    return countries

In [134]:
dict_countries = process_result_by_country("./Data/Early Results/","LeafAreaIndex")

data_summed = {}
for keys, values in dict_countries.items():
    data_summed[keys] = sum(values)
   
data_weighted = {}
max_key, max_value = max(data_summed.iteritems(), key=lambda x:x[1])

for keys, values in data_summed.items():
    data_weighted[keys] = values / max_value

In [132]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')

df.loc[:,'GDP (BILLIONS)'] = 0

for index, value in data_weighted.items():
    df.ix[df.CODE == index, 'GDP (BILLIONS)'] = value
    

In [123]:
data = [ dict(
        type = 'choropleth',
        locations = df['CODE'],
        z = df['GDP (BILLIONS)'],
        text = df['COUNTRY'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = 'Production Ratio ',
            title = 'Leaf Area Index'),
      ) ]

layout = dict(
    title = 'Simulated Cotton Production',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.sign_in('edcafenet', 'izgpN2YqqDk7tqDZGLEw')
py.iplot( fig, validate=False, filename='d3-world-map' )


High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~edcafenet/0 or inside your plot.ly account where it is named 'd3-world-map'
